# Item 25: Enforce Clarity with Keyword-Ony and Positional-Only Arguments

In [5]:
# The flexibity of keyword arguments enables us to write function that will be clear to new readers of our code
# for many use cases. For example, sya that we want to divide  one number by another but know that we need to be
# very careful about special cases
def safe_division(number, divisor, ignore_overflow, ignore_zero_division):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

In [6]:
# This call ignores the float overflow
result = safe_division(1.0, 10**500, True, False)
print(result)

0


In [7]:
# This call ignores division by zero
result = safe_division(1.0, 0, False, True)
print(result)

inf


The problem with the code above is that it's easy to confuse the position of the two Boolean arguments. This can cause bugs that are hard to track down. We can mitigate this by using keyword arguments.

In [19]:
def safe_division_b(number, divisor, 
                  ignore_overflow=False, # Changed
                  ignore_zero_division=False): # Changed
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

In [20]:
# Then, calers can use keyword arguments which of the ignore flags the want to set for specific operations,
# overriding the default behavior
result1 = safe_division_b(1.0, 10**500, ignore_overflow=True)
print(result1)

result2 = safe_division_b(1.0, 0, ignore_zero_division=True)
print(result2)

0
inf


The problem with `safe_division_b` is that there is nothing forcing callers to use keyword arguments (they are still optional). 

In [23]:
# Even with the new definition of safe_division_b, we can still call the function the old way with positional
# arguments
assert safe_division_b(1.0, 10**500, True, False) == 0

With complex functions like `safe_division` its better to require that callers are clear about their intentions by defining functions with *keyword-only* arguments. These arguments can only be supplied by keyword, never by position.

In [24]:
# Here we redefine the safe_division function to accept keyword-only arguments. The * symbol in the arguments
# list indicates the end of positional arguments and the beginning of keyword-only arguments


In [25]:
def safe_division_c(number, divisor, *,  # Changed
                  ignore_overflow=False,
                  ignore_zero_division=False):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

In [26]:
# Now calling the function with positional arguments for the keyword arguments won't work
safe_division_c(1.0, 10**500, True, False)

TypeError: safe_division_c() takes 2 positional arguments but 4 were given

In [30]:
# But keyword arguments and their default values will work as expected
result = safe_division_c(1.0, 0, ignore_zero_division=True)
assert result == float('inf')

try:
    result = safe_division_c(1.0, 0)
except ZeroDivisionError:
    pass # Expected

In [31]:
# However, a problem still remains with the safe_division_c version of this function: Callers may specify the
# first two required arguments (number and divisor) with a mix of positions and keywords
assert safe_division_c(number=2, divisor=5) == 0.4
assert safe_division_c(divisor=5,number=2) == 0.4
assert safe_division_c(2, divisor=5) == 0.4

In [33]:
# If we later decide to change the name of these two arguments for whatever reason, it will break break all
# subsequent calls to the function where number and divisor were specified using keyword arguments
def safe_division_c(numerator, denomitor, *,  # Changed
                  ignore_overflow=False,
                  ignore_zero_division=False):
    try:
        return numerator / denomitor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

safe_division_c(number=2, divisor=5)

TypeError: safe_division_c() got an unexpected keyword argument 'number'

The situation above is specifically problematic because we never intended for `number` and `divisor` to be part of an explicit interface for this function. These were just convinient parameters names that we chose for our implementation, and we didn't expect anyone to rely on them explicitly.

Thankfully, Python 3.8 introduces a solution to this problem, called *positional-only arguments*. These arguments shall be supplied only by position and never by keyword (the opposite of keyword-only arguments).

In [34]:
# Here we redefine the safe_division function to use positional-only arguments for the first two required
# parameters. The / symbol indicates where positional-only arguments end.
def safe_division_d(numerator, denomitor, /, *, # Changed
                  ignore_overflow=False,
                  ignore_zero_division=False):
    try:
        return numerator / denomitor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

In [35]:
# We can verify that this function works when the required arguments are provided positionally
assert safe_division_d(2, 5) == 0.4

In [36]:
# But an exception is raised if keyword arguments are used for the positional-only parameters
assert safe_division_d(numerator=2, denominator=5) == 0.4

TypeError: safe_division_d() got some positional-only arguments passed as keyword arguments: 'numerator'

One notable consequence of keyword-and positional-only argumentsis that any parameter names between the `/` and the `*` symbols in the arguments list may be passed by either position or by keyword (which is the default for all functions in Python). Depending on our API's styles and needs, allowing both types of arguments can increase readability and reduce noise. 

In [37]:
def safe_division_e(numerator, denomitor, /, 
                    ndigits=10, *, # Changed
                    ignore_overflow=False,
                    ignore_zero_division=False):
    try:
        fraction = numerator / denomitor # Changed
        return round(fraction, ndigits) # Changed
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

In [38]:
# Now we can call this new version of the function in all these different ways, since digits is an optional
# parameter that may be passed either by position or by keyword
result = safe_division_e(22, 7)
print(result)

result = safe_division_e(22, 7, 5)
print(result)

result = safe_division_e(22, 7, ndigits=2)
print(result)

3.1428571429
3.14286
3.14
